## Import Libraries

In [1]:
from sklearn.datasets import load_diabetes
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression
from utils.metric_utils import rmse_score
import pandas as pd
import numpy as np
import pickle

## Load model

* Muti-output Model

In [2]:
with open(f'../Model/catboost_all.pickle', 'rb') as f:
    model_muti_output = pickle.load(f)

* Single output Model

In [3]:
with open(f'../Model/catboost_single.pickle', 'rb' ) as f:
    single_output_model_dic = pickle.load(f)

* Chain model

In [4]:
with open('../model/chain_model_dic.pickle', 'rb') as f:
    chain_model_dic = pickle.load(f)

## load Data

In [5]:
with open(f'../Data/meta/train_x_after_feature_engineering.pickle', 'rb') as f:
    train_x_after_fe = pickle.load(f)

In [6]:
with open(f'../Data/meta/train_y_after_feature_engineering.pickle', 'rb') as f:
    train_y_after_fe = pickle.load(f)

In [7]:
# You need to use original values of target variable to evaluate RMSE
with open(f'../Data/meta/non_scaled_train_y_after_feature_engineering.pickle', 'rb') as f:
    non_scaled_train_y = pickle.load(f)

In [8]:
train_x_after_fe.head(3)

,clean_pressure11,clean_pressure23,clean_pressure31,clean_pressure41,clean_pressure51,clean_pressure52,clean_pressure61,clean_pressure62,clean_pressure72,clean_pressure81,...,oven_a1_group,oven_b3_group,painting_g2_act_f_air_group,painting_g4_act_hvc_group,painting_g10_act_hvc_group,painting_g11_act_hvc_group,env_rpi05_temp_group,env_rpi07_pm25_group,env_rpi14_pm1_group,env_rpi15_pm1_group
0,0.998661,-2.48405,-0.843686,2.359976,2.173607,-2.979055,1.913111,-0.765725,2.945531,1.949731,...,3,1,1,1,2,1,1,0,2,3
4,1.007741,-2.48645,-0.854122,2.359976,2.178187,-2.977987,1.913111,-0.774750,2.942140,1.821748,...,3,1,1,1,2,1,1,3,2,3
5,1.007741,-2.48645,-0.854122,2.359976,2.178187,-2.977987,1.913111,-0.774750,2.942140,1.821748,...,3,1,1,1,2,1,1,3,2,3


In [9]:
train_y_after_fe.head(3)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,-2.915553,-1.350983,-1.118309,-1.719137,-0.755270,-2.842387
4,-1.744669,-0.546567,-1.910943,-1.270337,-1.206139,-1.632568
5,-2.915553,-0.765953,-1.039045,-1.135697,-1.606911,-1.693059


In [10]:
non_scaled_train_y.head(3)

,sensor_point5_i_value,sensor_point6_i_value,sensor_point7_i_value,sensor_point8_i_value,sensor_point9_i_value,sensor_point10_i_value
0,42,56,82,41,73,45
4,57,78,62,51,64,65
5,42,72,84,54,56,64


In [11]:
# RandomForestRegressor(n_estimators=10, random_state=42)

In [12]:
stack_model_dic = {}
for col in train_y_after_fe.columns:
    estimators = [
        (f'{col}_model_1', single_output_model_dic[col]['model'])
    ]
    reg = StackingRegressor(
        estimators=estimators,
        final_estimator=XGBRegressor(n_estimators=10, random_state=42),
        n_jobs=2
    )

    reg.fit(train_x_after_fe, train_y_after_fe[col])
    stack_model_dic[col] = {'model': reg}

In [13]:
stack_model_dic[col]['model']

StackingRegressor(estimators=[('sensor_point10_i_value_model_1',
                               <catboost.core.CatBoostRegressor object at 0x147a42500>)],
                  final_estimator=XGBRegressor(base_score=None, booster=None,
                                               callbacks=None,
                                               colsample_bylevel=None,
                                               colsample_bynode=None,
                                               colsample_bytree=None,
                                               early_stopping_rounds=None,
                                               enable_categorical=False,
                                               eval_metric=None, gamma=None,
                                               gpu_id=None...y=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=None, max_bin=None,
                                               max_cat_to_onehot=None,
                                               max_delta_step=None,
                                               max_depth=None, max_leaves=None,
                                               min_child_weight=None,
                                               missing=nan,
                                               monotone_constraints=None,
                                               n_estimators=10, n_jobs=None,
                                               num_parallel_tree=None,
                                               predictor=None, random_state=42,
                                               reg_alpha=None, reg_lambda=None, ...),
                  n_jobs=2)

In [14]:
with open(f'../Model/target_col_list.pickle', 'rb' ) as f:
    col_y_li = pickle.load(f)
    
array_li= [ stack_model_dic[col_y]['model'].predict(train_x_after_fe).reshape(-1,1) for col_y in col_y_li ]
all_res = np.concatenate(array_li, axis=1)

In [15]:
# Load y-scaler model
with open(f'../Model/standard_scaler_y.pickle', 'rb') as f:
    scaler_y = pickle.load(f)

In [16]:
predictions = scaler_y.inverse_transform(all_res)
targets = non_scaled_train_y
print(rmse_score(predictions, targets))

sensor_point5_i_value     14.158172
sensor_point6_i_value     16.781120
sensor_point7_i_value     21.478309
sensor_point8_i_value     23.243289
sensor_point9_i_value     28.291124
sensor_point10_i_value    17.785837
dtype: float64
